In [3]:
import pandas as pd

df = pd.read_excel("files\GVC - Padrões Globais.xlsx")
df

<>:3: SyntaxWarning: invalid escape sequence '\G'
<>:3: SyntaxWarning: invalid escape sequence '\G'
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_16460\3371848847.py:3: SyntaxWarning: invalid escape sequence '\G'
  df = pd.read_excel("files\GVC - Padrões Globais.xlsx")


,Código,Termo
0,3E 02 00 00 00 00 00,Elementos de manejo do terreno
1,3E 02 02 00 00 00 00,Elementos de terrapleno
2,3E 02 02 02 00 00 00,Solo
3,3E 02 02 04 00 00 00,Camada de solo orgânico
4,3E 02 02 06 00 00 00,Platô
...,...,...
1012,3R 22 06 02 00 00 00,treinado (a)
1013,3R 22 06 06 00 00 00,capacitado (a)
1014,3R 22 10 00 00 00 00,Resultados dos trabalhos de segurança e saúde
1015,3R 22 10 02 00 00 00,sinalizado (a)


In [4]:
codeTermDict: dict[str, str] = {}

for idx, value in df["Código"].items():
    parts = value.split()
    
    while len(parts) < 7:
        parts.append("00")
    
    newValue = "-".join(parts)  # '3E-02-00-00-00-00-00'
    if "3R" in newValue:
        continue

    term = df.at[idx, "Termo"]
    codeTermDict[newValue] = term

import json
import os

os.makedirs("files", exist_ok=True)
file_path = os.path.join("files", "codes.json")

with open(file_path, "w", encoding="utf-8") as f:
    json.dump(codeTermDict, f, indent=2, ensure_ascii=False)

codeTermDict

{'3E-02-00-00-00-00-00': 'Elementos de manejo do terreno',
 '3E-02-02-00-00-00-00': 'Elementos de terrapleno',
 '3E-02-02-02-00-00-00': 'Solo',
 '3E-02-02-04-00-00-00': 'Camada de solo orgânico',
 '3E-02-02-06-00-00-00': 'Platô',
 '3E-02-02-08-00-00-00': 'Faixa terraplanada',
 '3E-02-04-00-00-00-00': 'Elementos de contenção',
 '3E-02-04-02-00-00-00': 'Muro de arrimo',
 '3E-02-04-02-02-00-00': 'Muro de arrimo por gravidade',
 '3E-02-04-02-04-00-00': 'Muro de arrimo por flexão',
 '3E-02-04-02-06-00-00': 'Muro de gabião',
 '3E-02-04-02-08-00-00': 'Muro de flexão',
 '3E-02-04-04-00-00-00': 'Solo grampeado',
 '3E-02-04-06-00-00-00': 'Cortina de contenção',
 '3E-02-04-06-02-00-00': 'Contenção escorada',
 '3E-02-04-06-04-00-00': 'Contenção não escorada',
 '3E-02-04-06-06-00-00': 'Berma',
 '3E-02-04-06-08-00-00': 'Cortina de contenção com pranchão',
 '3E-02-04-06-10-00-00': 'Cortina de contenção com estacas justapostas',
 '3E-02-04-06-12-00-00': 'Cortina de contenção com parede diafragma',
 '3